In [1]:
import pandas as pd
#import pyspark as ps
import numpy as np
import matplotlib.pyplot as plt

# Data Extraction

In [ ]:
#If you want to extract the dataset directly from kaggle to collab

#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets download -d jeetahirwar/northwind-traders

In [ ]:
#!unzip /content/northwind-traders.zip -d /content/drive/MyDrive/northwind

In [4]:
customers=pd.read_csv("Northwind_csv\csv\customers.csv",encoding="ISO-8859-1")
employees=pd.read_csv("Northwind_csv\csv\employees.csv",encoding="ISO-8859-1")
categories=pd.read_csv("Northwind_csv\csv\categories.csv",encoding="ISO-8859-1")
order_details=pd.read_csv("Northwind_csv\csv\order_details.csv",encoding="ISO-8859-1")
orders=pd.read_csv("Northwind_csv\csv\orders.csv",encoding="ISO-8859-1")
products=pd.read_csv("Northwind_csv\csv\products.csv",encoding="ISO-8859-1")
shippers=pd.read_csv("Northwind_csv\csv\shippers.csv",encoding="ISO-8859-1")

# Exploration of the tables

In [47]:
customers.head()

,customerID,companyName,contactName,contactTitle,city,country
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Berlin,Germany
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Mexico City,Mexico
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mexico City,Mexico
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,London,UK
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Luleå,Sweden


In [48]:
employees.head()

,employeeID,employeeName,title,city,country,reportsTo
0,1,Nancy Davolio,Sales Representative,New York,USA,8.0
1,2,Andrew Fuller,Vice President Sales,New York,USA,NaN
2,3,Janet Leverling,Sales Representative,New York,USA,8.0
3,4,Margaret Peacock,Sales Representative,New York,USA,8.0
4,5,Steven Buchanan,Sales Manager,London,UK,2.0


In [49]:
categories.head()

,categoryID,categoryName,description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains & Cereals,"Breads, crackers, pasta, and cereal"


In [50]:
order_details.head()

,orderID,productID,unitPrice,quantity,discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


In [51]:
orders.head()

,orderID,customerID,employeeID,orderDate,requiredDate,shippedDate,shipperID,freight
0,10248,VINET,5,2013-07-04,2013-08-01,2013-07-16,3,32.38
1,10249,TOMSP,6,2013-07-05,2013-08-16,2013-07-10,1,11.61
2,10250,HANAR,4,2013-07-08,2013-08-05,2013-07-12,2,65.83
3,10251,VICTE,3,2013-07-08,2013-08-05,2013-07-15,1,41.34
4,10252,SUPRD,4,2013-07-09,2013-08-06,2013-07-11,2,51.30


In [52]:
products.head()

,productID,productName,quantityPerUnit,unitPrice,discontinued,categoryID
0,1,Chai,10 boxes x 20 bags,18.00,0,1
1,2,Chang,24 - 12 oz bottles,19.00,0,1
2,3,Aniseed Syrup,12 - 550 ml bottles,10.00,0,2
3,4,Chef Anton's Cajun Seasoning,48 - 6 oz jars,22.00,0,2
4,5,Chef Anton's Gumbo Mix,36 boxes,21.35,1,2


In [53]:
shippers.head()

,shipperID,companyName
0,1,Speedy Express
1,2,United Package
2,3,Federal Shipping


# Creation of the time dimension :

In [3]:
time =pd.DataFrame(orders['orderDate'].map(lambda x : x[:-3]).unique()).reset_index()
time.columns= ['timeID','year']
time['month']=time['year'].astype('str').map(lambda x: x[-2:])
time['timeID']=time.year.map(lambda x: x[2:4]+x[-2:])
time['year']=time['year'].map(lambda x : x[:-3])
time.head()

,timeID,year,month
0,1307,2013,07
1,1308,2013,08
2,1309,2013,09
3,1310,2013,10
4,1311,2013,11


###Data Transformation

In [5]:
#Map Function to get the shipped orders
def shipped(x) :
  return type(x)==str

In [6]:
nb_orders=orders.loc[:,['orderID','customerID','employeeID','orderDate','shippedDate']]
nb_orders["orderDate"]=nb_orders["orderDate"].map(lambda x : x[:-3])
#Unique timeID for each date :
nb_orders['timeID']=nb_orders['orderDate'].map(lambda x: x[2:4]+x[-2:])
#Droping unnecessary columns for the process
nb_orders=nb_orders.drop(['orderID','orderDate'],axis=1)
#Finding the shipped orders
nb_orders.shippedDate=nb_orders.shippedDate.map(lambda x : shipped(x))
nb_orders=nb_orders.rename(columns={'shippedDate':'NB_shipped'})
#Reordering the columns
nb_orders = nb_orders[['customerID','employeeID','timeID','NB_shipped']]

nb_orders['NB_unshipped']=nb_orders['NB_shipped'].map(lambda x : not x)

#Grouping by the primary key  (customerID*,employeeID*,timeID*) and calculate the sum aggragation values

nb_orders=nb_orders.groupby(['customerID','employeeID','timeID']).agg('sum').reset_index(['customerID','employeeID','timeID'])
nb_orders


#hadi tableau

# Another datawarehouse on :
# Customers, qte , price ...
#hadi powerbi

,customerID,employeeID,timeID,NB_shipped,NB_unshipped
0,ALFKI,1,1501,1,0
1,ALFKI,1,1503,1,0
2,ALFKI,3,1504,1,0
3,ALFKI,4,1410,2,0
4,ALFKI,6,1408,1,0
...,...,...,...,...,...
801,WOLZA,4,1502,1,0
802,WOLZA,4,1504,1,0
803,WOLZA,5,1502,1,0
804,WOLZA,6,1407,1,0


##**Loading Data in our database**

# Connection to the Database

In [7]:
#!pip install pypyodbc
#!pip install sqlserver
#!pip install pymysql

In [33]:
# !curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
# !curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
# !sudo apt-get update
# !sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17
#!sudo apt-get install unixodbc-dev
#!pip install chart_studio

In [5]:
#!pip install pyodbc

In [8]:
import pyodbc as odbc
#from sqlalchemy import create_engine
import pymysql
conn = odbc.connect('Driver={SQL Server};'
                        'Server=DESKTOP-PHIS9EP\SQLEXPRESS;'
                        'Database=Northwind;'
                        'Trusted_Connection=yes;')

cursor = conn.cursor()

In [9]:
conn

# Loading the processed data into our datawarehouse :

In [10]:
cursor.execute('TRUNCATE TABLE CUSTOMER')

In [15]:
#Fact table loading
for index , row in nb_orders.iterrows():
    c= row.customerID
    e =row.employeeID
    t= row.timeID
    ns=row.NB_shipped
    nu=row.NB_unshipped
    #print(type(row['customerID']))
    #cursor.execute("INSERT INTO NB_ORDER VALUES ('"+row['customerID']+"',"+str(row['employeeID'])+","+str(row['timeID'])+","+str(row['NB_shipped'])+","+str(row['NB_unshipped']+");")
    cursor.execute("INSERT INTO NB_ORDER VALUES('"+c+"',"+str(e)+","+str(t)+","+str(ns)+","+str(nu)+")")     
    
conn.commit()

In [11]:
#Some modifications on the tables

#cursor.execute('''
#--ALTER TABLE CUSTOMER
#--DROP CONSTRAINT C1
#--ALTER TABLE CUSTOMER
#--ALTER COLUMN CUSTOMER_ID VARCHAR(50) NOT NULL 
#ALTER TABLE CUSTOMER
#ALTER COLUMN COMPANY_NAME VARCHAR(50) NOT NULL 
#--ALTER TABLE CUSTOMER ADD CONSTRAINT C1 PRIMARY KEY (CUSTOMER_ID)
#''')
#cursor.execute('TRUNCATE TABLE CUSTOMER')

In [24]:
#Customer table loading :
for index , row in customers.iterrows():
    cust=row['customerID']
    cmp=row['companyName'].replace("'","''")
    city=row['city']
    country=row['country']
    x= f" INSERT INTO CUSTOMER VALUES ('{str(cust)}','{str(cmp)}','{city}','{country}') "
    #print(x)
    cursor.execute(x)
conn.commit()

In [17]:
#Time table :
for index , row in time.iterrows(): 
    time_id=row['timeID']
    y= row['year']
    m = row['month']
    x= f"INSERT INTO TIME VALUES ({time_id},{m},{y})"
    cursor.execute(x)
conn.commit()

In [12]:
cursor.execute('SELECT * FROM TIME').fetchall()

[(1307, 7, 2013),
 (1308, 8, 2013),
 (1309, 9, 2013),
 (1310, 10, 2013),
 (1311, 11, 2013),
 (1312, 12, 2013),
 (1401, 1, 2014),
 (1402, 2, 2014),
 (1403, 3, 2014),
 (1404, 4, 2014),
 (1405, 5, 2014),
 (1406, 6, 2014),
 (1407, 7, 2014),
 (1408, 8, 2014),
 (1409, 9, 2014),
 (1410, 10, 2014),
 (1411, 11, 2014),
 (1412, 12, 2014),
 (1501, 1, 2015),
 (1502, 2, 2015),
 (1503, 3, 2015),
 (1504, 4, 2015),
 (1505, 5, 2015)]

In [18]:
#Employees table :
for index ,row in employees.iterrows() :
    id_ = row['employeeID']
    name = row['employeeName']
    city = row['city']
    cnt = row['country']
    
    x= f"INSERT INTO EMPLOYEE VALUES ({id_},'{name}','{city}','{cnt}')"
    cursor.execute(x)
conn.commit()

In [19]:
#cursor.execute('''
#ALTER TABLE TIME 
#ADD CONSTRAINT C4 PRIMARY KEY (TIME_ID)
#''')

In [13]:
cursor.execute('SELECT * FROM NB_order').fetchall()

[('ALFKI', 1, 1501, 1, 0),
 ('ALFKI', 1, 1503, 1, 0),
 ('ALFKI', 3, 1504, 1, 0),
 ('ALFKI', 4, 1410, 2, 0),
 ('ALFKI', 6, 1408, 1, 0),
 ('ANATR', 3, 1408, 1, 0),
 ('ANATR', 3, 1411, 1, 0),
 ('ANATR', 4, 1503, 1, 0),
 ('ANATR', 7, 1309, 1, 0),
 ('ANTON', 1, 1409, 1, 0),
 ('ANTON', 3, 1311, 1, 0),
 ('ANTON', 3, 1409, 1, 0),
 ('ANTON', 3, 1501, 1, 0),
 ('ANTON', 4, 1405, 1, 0),
 ('ANTON', 7, 1404, 1, 0),
 ('ANTON', 7, 1406, 1, 0),
 ('AROUT', 1, 1402, 1, 0),
 ('AROUT', 1, 1406, 1, 0),
 ('AROUT', 1, 1411, 1, 0),
 ('AROUT', 3, 1412, 2, 0),
 ('AROUT', 4, 1410, 1, 0),
 ('AROUT', 4, 1411, 1, 0),
 ('AROUT', 4, 1502, 1, 0),
 ('AROUT', 4, 1503, 1, 0),
 ('AROUT', 6, 1311, 1, 0),
 ('AROUT', 8, 1312, 1, 0),
 ('AROUT', 9, 1503, 1, 0),
 ('AROUT', 9, 1504, 1, 0),
 ('BERGS', 1, 1405, 1, 0),
 ('BERGS', 1, 1408, 1, 0),
 ('BERGS', 1, 1410, 1, 0),
 ('BERGS', 1, 1411, 1, 0),
 ('BERGS', 2, 1308, 1, 0),
 ('BERGS', 3, 1312, 1, 0),
 ('BERGS', 3, 1402, 2, 0),
 ('BERGS', 3, 1406, 1, 0),
 ('BERGS', 3, 1412, 1, 0),
 

In [14]:
cursor.close()
conn.close()

In [27]:
cursor.execute('SELECT * FROM EMPLOYEE').fetchall()

[(1, 'Nancy Davolio', 'New York', 'USA'),
 (2, 'Andrew Fuller', 'New York', 'USA'),
 (3, 'Janet Leverling', 'New York', 'USA'),
 (4, 'Margaret Peacock', 'New York', 'USA'),
 (5, 'Steven Buchanan', 'London', 'UK'),
 (6, 'Michael Suyama', 'London', 'UK'),
 (7, 'Robert King', 'London', 'UK'),
 (8, 'Laura Callahan', 'New York', 'USA'),
 (9, 'Anne Dodsworth', 'London', 'UK')]